## 1. Постановка задачи

<center> <img src=https://storage.googleapis.com/kaggle-competitions/kaggle/3333/media/taxi_meter.png align="right" width="300"/> </center>
    
Вам предстоит решить настоящую задачу машинного обучения, направленную на автоматизацию бизнес процессов. Мы построим модель, которая будет предсказывать общую продолжительность поездки такси в Нью-Йорке. 

Представьте вы заказываете такси из одной точки Нью-Йорка в другую, причем не обязательно конечная точка должна находиться в пределах города. Сколько вы должны будете за нее заплатить? Известно, что стоимость такси в США  рассчитывается на основе фиксированной ставки + тарифная стоимость, величина которой зависит от времени и расстояния. Тарифы варьируются в зависимости от города.

В свою очередь время поездки зависит от множества факторов таких как, откуда и куда вы едете, в какое время суток вы совершаете вашу поездку, погодных условий и так далее. 

Таким образом, если мы разработаем алгоритм, способный определять длительность поездки, мы сможем прогнозировать ее стоимость самым тривиальным образом, например, просто умножая стоимость на заданный тариф. 
Сервисы такси хранят огромные объёмы информации о поездках, включая такие данные как конечная, начальная точка маршрута, дата поездки и ее длительность. Эти данные можно использовать для того, чтобы прогнозировать длительность поездки в автоматическом режиме с привлечением искусственного интеллекта.

**Бизнес-задача:** определить характеристики и с их помощью спрогнозировать длительность поездки такси.

**Техническая задача для вас как для специалиста в Data Science:** построить модель машинного обучения, которая на основе предложенных характеристик клиента будет предсказывать числовой признак - время поездки такси. То есть решить задачу регрессии.

**Основные цели проекта:**
1. Сформировать набор данных на основе нескольких источников информации
2. Спроектировать новые признаки с помощью Feature Engineering и выявить наиболее значимые при построении модели
3. Исследовать предоставленные данные и выявить закономерности
4. Построить несколько моделей и выбрать из них наилучшую по заданной метрике
5. Спроектировать процесс предсказания времени длительности поездки для новых данных

Загрузить свое решение на платформу Kaggle, тем самым поучаствовав в настоящем Data Science соревновании.
Во время выполнения проекта вы отработаете навыки работы с несколькими источниками данных, генерации признаков, разведывательного анализа и визуализации данных, отбора признаков и, конечно же, построения моделей машинного обучения!


## 2. Знакомство с данными, базовый анализ и расширение данных

Начнём наше исследование со знакомства с предоставленными данными. А также подгрузим дополнительные источники данных и расширим наш исходный датасет. 


Заранее импортируем модули, которые нам понадобятся для решения задачи:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

Прочитаем наш файл с исходными данными:

In [ ]:
taxi_data = pd.read_csv("/Users/Anna/Study/skillfactory/practice/math/train.csv")
print('Train data shape: {}'.format(taxi_data.shape))
taxi_data.head()

Итак, у нас с вами есть данные о почти 1.5 миллионах поездок и 11 характеристиках, которые описывают каждую из поездок. 

Мы условно разделили признаки нескольких групп. Каждой из групп мы в дальнейшем уделим отдельное внимание.

**Данные о клиенте и таксопарке:**
* id - уникальный идентификатор поездки
* vendor_id - уникальный идентификатор поставщика (таксопарка), связанного с записью поездки

**Временные характеристики:**
* pickup_datetime - дата и время, когда был включен счетчик поездки
* dropoff_datetime - дата и время, когда счетчик был отключен

**Географическая информация:**
* pickup_longitude -  долгота, на которой был включен счетчик
* pickup_latitude - широта, на которой был включен счетчик
* dropoff_longitude - долгота, на которой счетчик был отключен
* dropoff_latitude - широта, на которой счетчик был отключен

**Прочие признаки:**
* passenger_count - количество пассажиров в транспортном средстве (введенное водителем значение)
* store_and_fwd_flag - флаг, который указывает, сохранилась ли запись о поездке в памяти транспортного средства перед отправкой поставщику. Y - хранить и пересылать, N - не хранить и не пересылать поездку.

**Целевой признак:**
* trip_duration - продолжительность поездки в секундах


Для начала мы проведем базовый анализ того, насколько данные готовы к дальнейшей предобработке и анализу. 

### Задание 2.1
Для начала посмотрим на временные рамки, в которых мы работаем с данными.

Переведите признак pickup_datetime в тип данных datetime с форматом год-месяц-день час:минута:секунда (в функции pd.to_datetime() параметр format='%Y-%m-%d %H:%M:%S'). 

Определите временные рамки (без учета времени), за которые представлены данные.

In [ ]:
# ваш код здесь
taxi_data['pickup_datetime'] = pd.to_datetime(taxi_data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
taxi_data['pickup_date'] = taxi_data['pickup_datetime'].dt.date

In [ ]:
plot = taxi_data.groupby('pickup_date')['pickup_date'].value_counts()
fig = px.line(plot)
fig.show()

### Задание 2.2
Посмотрим на пропуски. 
Сколько пропущенных значений присутствует в данных (суммарно по всем столбцам таблицы)?

In [ ]:
# ваш код здесь
taxi_data.isnull().sum()

In [ ]:
taxi_data.describe()

In [ ]:
taxi_data[taxi_data['passenger_count']==0]['passenger_count'].count()

### Задание 2.3
Посмотрим на статистические характеристики некоторых признаков. 

а) Сколько уникальных таксопарков присутствует в данных?

б) Каково максимальное количество пассажиров?

в) Чему равна средняя и медианная длительность поездки? Ответ приведите в секундах и округлите до целого.

г) Чему равно минимальное и максимальное время поездки (в секундах)?


In [ ]:
# ваш код здесь
#а) Сколько уникальных таксопарков присутствует в данных?
v_id = taxi_data['vendor_id'].nunique()
print('number of unique taxi parks:', v_id)

#б) Каково максимальное количество пассажиров?
pas = taxi_data['passenger_count'].max()
print('maximum number of passengers:', pas)
#в) Чему равна средняя и медианная длительность поездки? Ответ приведите в секундах и округлите до целого.
mean = round(taxi_data['trip_duration'].mean())
median = round(taxi_data['trip_duration'].median())
print('mean and median time of a trip - {} and {}'.format(mean, median))
#г) Чему равно минимальное и максимальное время поездки (в секундах)?
min = taxi_data['trip_duration'].min()
max = taxi_data['trip_duration'].max()
print('minimal and maximum time of a trip - {} and {}'.format(min, max))

Займемся расширением исходного набора данных как с помощью внешних источников, так и с помощью манипуляций над имеющимися в данных признаками. 


### Задание 2.4
Реализуйте функцию add_datetime_features(), которая принимает на вход таблицу с данными о поездках (DataFrame) и возвращает ту же таблицу с добавленными в нее 3 столбцами:
* pickup_date - дата включения счетчика - начала поездки (без времени);
* pickup_hour - час дня включения счетчика;
* pickup_day_of_week - наименование дня недели, в который был включен счетчик.

а) Сколько поездок было совершено в субботу?

б) Сколько поездок в среднем совершается в день? Ответ округлите до целого

In [ ]:
taxi_data['pickup_datetime'].dt.day

In [ ]:
# ваш код здесь
def add_datetime_features(df):
    df['pickup_hour'] = df['pickup_datetime'].dt.hour
    df['pickup_day_of_week'] = df['pickup_datetime'].dt.day_name()
    return df

taxi_data = add_datetime_features(taxi_data)
display(taxi_data.info())
display(taxi_data.head(2))

In [ ]:
#а) Сколько поездок было совершено в субботу?
sat = taxi_data[taxi_data['pickup_day_of_week']=='Saturday']['pickup_day_of_week'].count()
print('number of saturday trips:', sat)
#б) Сколько поездок в среднем совершается в день? Ответ округлите до целого
mean = round(taxi_data['pickup_date'].value_counts().mean())
print('mean number of trips per day', mean)

### Задание 2.5
Реализуйте функцию add_holiday_features(), которая принимает на вход две таблицы: 
* таблицу с данными о поездках;
* таблицу с данными о праздничных днях;

и возвращает обновленную таблицу с данными о поездках с добавленным в нее столбцом pickup_holiday - бинарным признаком того, начата ли поездка в праздничный день или нет (1 - да, 0 - нет). 

Чему равна медианная длительность поездки на такси в праздничные дни? Ответ приведите в секундах, округлив до целого.


In [ ]:
# ваш код здесь
holiday_data = pd.read_csv('/Users/Anna/Study/skillfactory/practice/math/holiday_data.csv', sep=';')
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
display(holiday_data.head(2))
display(holiday_data.info())

In [ ]:
taxi_data['pickup_date'] = pd.to_datetime(taxi_data['pickup_date'])
taxi_data.merge(holiday_data, how='left', left_on='pickup_date', right_on='date')['holiday'].value_counts()

In [ ]:
def add_holiday_features(df1, df2):
    df = df1.merge(df2, how='left', left_on='pickup_date', right_on='date')
    df.drop(columns=['day','date'], inplace = True)
    df['holiday'].fillna(0, inplace=True)
    df.loc[df['holiday'] != 0, 'holiday'] = 1
    df.rename(columns={'holiday':'pickup_holiday'}, inplace=True)
    return df 
taxi_data = add_holiday_features(taxi_data,holiday_data)
display(taxi_data.info())
display(taxi_data.head(2))      

In [ ]:
taxi_data.isna().sum()

In [ ]:
taxi_data['pickup_holiday'].value_counts()

In [ ]:
#медианная длительность поездки на такси в праздничные дни
round(taxi_data[taxi_data['pickup_holiday']==1]['trip_duration'].median())

### Задание 2.6
Реализуйте функцию add_osrm_features(), которая принимает на вход две таблицы:
* таблицу с данными о поездках;
* таблицу с данными из OSRM;

и возвращает обновленную таблицу с данными о поездках с добавленными в нее 3 столбцами:
* total_distance;
* total_travel_time;
* number_of_steps.

а) Чему равна разница (в секундах) между медианной длительностью поездки в данных и медианной длительностью поездки, полученной из OSRM? 

В результате объединения таблиц у вас должны были получиться пропуски в столбцах с информацией из OSRM API. Это связано с тем, что для некоторых поездок не удалось выгрузить данные из веб источника. 

б) Сколько пропусков содержится в столбцах с информацией из OSRM API после объединения таблиц?

In [ ]:
# ваш код здесь
osrm_data = pd.read_csv('/Users/Anna/Study/skillfactory/practice/math/osrm_data_train.csv')
osrm_data.head(2)

In [ ]:
def add_osrm_features(df1,df2):
    df2=df2[['id','total_distance', 'total_travel_time', 'number_of_steps']]
    df = df1.merge(df2, how='left', on='id')
    return df

taxi_data = add_osrm_features(taxi_data,osrm_data)
taxi_data.head(2) 
    

In [ ]:
#а) разница (в секундах) между медианной длительностью поездки в данных и медианной длительностью поездки, полученной из OSRM? 

dif = taxi_data['trip_duration'].median() - taxi_data['total_travel_time'].median()
print('difference in median trip duration btw 2 df:', dif)

#б) Сколько пропусков содержится в столбцах с информацией из OSRM API после объединения таблиц?
mask = ['total_distance', 'total_travel_time', 'number_of_steps']
taxi_data.isna().sum()

In [ ]:
def get_haversine_distance(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # радиус земли в километрах
    EARTH_RADIUS = 6371 
    # считаем кратчайшее расстояние h по формуле Хаверсина
    lat_delta = lat2 - lat1
    lng_delta = lng2 - lng1
    d = np.sin(lat_delta * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng_delta * 0.5) ** 2
    h = 2 * EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def get_angle_direction(lat1, lng1, lat2, lng2):
    # переводим углы в радианы
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    # считаем угол направления движения alpha по формуле угла пеленга
    lng_delta_rad = lng2 - lng1
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    alpha = np.degrees(np.arctan2(y, x))
    return alpha

### Задание 2.7.
Реализуйте функцию add_geographical_features(), которая принимает на вход таблицу с данными о поездках и возвращает обновленную таблицу с добавленными в нее 2 столбцами:
* haversine_distance - расстояние Хаверсина между точкой, в которой был включен счетчик, и точкой, в которой счетчик был выключен;
* direction - направление движения из точки, в которой был включен счетчик, в точку, в которой счетчик был выключен.

Чему равно медианное расстояние Хаверсина поездок (в киллометрах)? Ответ округлите до сотых.


In [ ]:
# ваш код здесь
def add_geographical_features(df, inplace=True):
    lat1 = df['pickup_latitude']
    lng1 = df['pickup_longitude']
    lat2 = df['dropoff_latitude']
    lng2 = df['dropoff_longitude']
    df['haversine_distance'] = get_haversine_distance(lat1, lng1, lat2, lng2)
    df['direction'] = get_angle_direction(lat1, lng1, lat2, lng2)
    return df

add_geographical_features(taxi_data, inplace=True)


In [ ]:
#медианное расстояние Хаверсина поездок (в киллометрах)
round(taxi_data['haversine_distance'].median(),2)

### Задание 2.8.
Реализуйте функцию add_cluster_features(), которая принимает на вход таблицу с данными о поездках и обученный алгоритм кластеризации. Функция должна возвращать обновленную таблицу с добавленными в нее столбцом geo_cluster - географический кластер, к которому относится поездка.

Сколько поездок содержится в наименьшем по размеру географическом кластере?


In [ ]:
coords = np.hstack((taxi_data[['pickup_latitude', 'pickup_longitude']],
                    taxi_data[['dropoff_latitude', 'dropoff_longitude']]))

In [ ]:
#pip install --upgrade threadpoolctl

In [ ]:
# создаем обучающую выборку из географических координат всех точек
coords = np.hstack((taxi_data[['pickup_latitude', 'pickup_longitude']],
                    taxi_data[['dropoff_latitude', 'dropoff_longitude']]))
# обучаем алгоритм кластеризации
kmeans = cluster.KMeans(n_clusters=10, random_state=42)
kmeans.fit(coords)

In [ ]:
# ваш код здесь
def add_cluster_features(df, model):
    coords = np.hstack((df[['pickup_latitude', 'pickup_longitude']],
                    df[['dropoff_latitude', 'dropoff_longitude']]))
    df['geo_cluster'] = model.predict(coords)
    return df
add_cluster_features(taxi_data, kmeans)
display(taxi_data.head(2))
display(taxi_data.info())

In [ ]:
# Get the cluster labels for each data point
cluster_labels = kmeans.predict(coords)

# Get the cluster centers (centroids)
cluster_centers = kmeans.cluster_centers_

# Calculate the squared distances from each data point to its assigned cluster centroid
squared_distances = np.sum((coords - cluster_centers[cluster_labels]) ** 2, axis=1)

# Add the squared distances as a new column in your DataFrame
taxi_data['cluster_squared_distance'] = squared_distances

In [ ]:
plot = taxi_data.groupby('geo_cluster').agg({'cluster_squared_distance': 'mean', 'id': 'count'}).reset_index()
plot.sort_values('cluster_squared_distance')

### Задание 2.9.
Реализуйте функцию add_weather_features(), которая принимает на вход две таблицы:
* таблицу с данными о поездках;
* таблицу с данными о погодных условиях на каждый час;

и возвращает обновленную таблицу с данными о поездках с добавленными в нее 5 столбцами:
* temperature - температура;
* visibility - видимость;
* wind speed - средняя скорость ветра;
* precip - количество осадков;
* events - погодные явления.

а) Сколько поездок было совершено в снежную погоду?

В результате объединения у вас должны получиться записи, для которых в столбцах temperature, visibility, wind speed, precip, и events будут пропуски. Это связано с тем, что в таблице с данными о погодных условиях отсутствуют измерения для некоторых моментов времени, в которых включался счетчик поездки. 

б) Сколько процентов от общего количества наблюдений в таблице с данными о поездках занимают пропуски в столбцах с погодными условиями? Ответ приведите с точностью до сотых процента.


In [ ]:
# ваш код здесь
weather_data = pd.read_csv('/Users/Anna/Study/skillfactory/practice/math/weather_data.csv')
weather_data['date'] = pd.to_datetime(weather_data['date'])
display(weather_data.head(2))
display(weather_data.info())


In [ ]:
def add_weather_features(df1,df2):
    df2 = df2[['temperature','visibility',
                          'wind speed','precip',
                          'events', 'date', 'hour']]
    df = df1.merge(df2, how='left', left_on=['pickup_date', 'pickup_hour'], 
                                             right_on = ['date', 'hour'])
    df.drop(columns = ['date', 'hour'], inplace=True)
    return df
taxi_data = add_weather_features(taxi_data,weather_data)
display(taxi_data.head(2))
display(taxi_data.info())

In [ ]:
#а) Сколько поездок было совершено в снежную погоду?
taxi_data['events'].value_counts()

In [ ]:
#б) Сколько процентов от общего количества наблюдений в таблице с данными о поездках занимают пропуски в столбцах с погодными условиями?
display(taxi_data.isna().sum())
per = 12032/1458643*100
per

### Задание 2.10.
Реализуйте функцию fill_null_weather_data(), которая принимает на вход которая принимает на вход таблицу с данными о поездках. Функция должна заполнять пропущенные значения в столбцах.

Пропуски в столбцах с погодными условиями -  temperature, visibility, wind speed, precip заполните медианным значением температуры, влажности, скорости ветра и видимости в зависимости от даты начала поездки. Для этого сгруппируйте данные по столбцу pickup_date и рассчитайте медиану в каждой группе, после чего с помощью комбинации методов transform() и fillna() заполните пропуски. 
Пропуски в столбце events заполните строкой 'None' - символом отсутствия погодных явлений (снега/дождя/тумана). 

Пропуски в столбцах с информацией из OSRM API - total_distance, total_travel_time и number_of_steps заполните медианным значением по столбцам. 

Чему равна медиана в столбце temperature после заполнения пропусков? Ответ округлите до десятых.


In [ ]:
# ваш код здесь
def fill_null_weather_data(df):
   value = {'temperature':df.groupby('pickup_date')['temperature'].transform('mean'),
       'visibility':df.groupby('pickup_date')['visibility'].transform('mean'),
       'wind speed':df.groupby('pickup_date')['wind speed'].transform('mean'),
        'precip':df.groupby('pickup_date')['precip'].transform('mean'),
        'events':'None',
        'total_distance':df['total_distance'].median(),
        'total_travel_time':df['total_travel_time'].median(),
        'number_of_steps':df['number_of_steps'].median()}
   df.fillna(value, inplace=True)

fill_null_weather_data(taxi_data)
display(taxi_data.isna().sum())
#медиана в столбце temperature
taxi_data['temperature'].median()

В завершение первой части найдем очевидные выбросы в целевой переменной - длительности поездки. 

Проще всего найти слишком продолжительные поездки. Давайте условимся, что выбросами будут считаться поездки, длительность которых превышает 24 часа. 

Чуть сложнее с анализом поездок, длительность которых слишком мала. Потому что к ним относятся действительно реальные поездки на короткие расстояния, поездки, которые были отменены через секунду после того как включился счетчик, а также “телепортации” - перемещение на большие расстояния за считанные секунды. 
Условимся, что мы будем считать выбросами только последнюю группу. Как же нам их обнаружить наиболее простым способом?

Можно воспользоваться информацией о кратчайшем расстоянии, которое проезжает такси. Вычислить среднюю скорость автомобиля на кратчайшем пути следующим образом: 
$$avg\_speed= \frac{total\_distance}{1000*trip\_duration}*3600$$
Если мы построим диаграмму рассеяния средней скорости движения автомобилей, мы увидим следующую картину:


In [ ]:
avg_speed = taxi_data['total_distance'] / taxi_data['trip_duration'] * 3.6
fig, ax = plt.subplots(figsize=(10, 5))
sns.scatterplot(x=avg_speed.index, y=avg_speed, ax=ax)
ax.set_xlabel('Index')
ax.set_ylabel('Average speed');

Как раз отсюда мы видим, что у нас есть “поездки-телепортации”, для которых средняя скорость более 1000 км/ч. Даже есть такая, средняя скорость которой составляла более 12000 км/ч! 

Давайте условимся, что предельная средняя скорость, которую могут развивать таксисты будет 300 км/ч. 


### Задание 2.11.
Найдите поездки, длительность которых превышает 24 часа. И удалите их из набора данных.

а) Сколько выбросов по признаку длительности поездки вам удалось найти?

Найдите поездки, средняя скорость которых по кратчайшему пути превышает 300 км/ч и удалите их из данных. 

б) Сколько выбросов по признаку скорости вам удалось найти?

In [ ]:
# ваш код здесь
display(taxi_data[taxi_data['trip_duration']/60/60 > 24]['trip_duration'].count())
taxi_data['average speed'] = taxi_data['total_distance'] / taxi_data['trip_duration'] * 3.6
display(taxi_data[taxi_data['average speed'] > 300]['average speed'].count())


In [ ]:
taxi_data = taxi_data[taxi_data['trip_duration']/60/60 <= 24]
taxi_data = taxi_data[taxi_data['average speed'] <= 300]

In [ ]:
display(taxi_data.head(2))
display(taxi_data.info())

## 3. Разведывательный анализ данных (EDA)

В этой части нашего проекта мы с вами:
* Исследуем сформированный набор данных; 
* Попробуем найти закономерности, позволяющие сформулировать предварительные гипотезы относительно того, какие факторы являются решающими в определении длительности поездки;
* Дополним наш анализ визуализациями, иллюстрирующими; исследование. Постарайтесь оформлять диаграммы с душой, а не «для галочки»: навыки визуализации полученных выводов обязательно пригодятся вам в будущем.


Начинаем с целевого признака. Забегая вперед, скажем, что основной метрикой качества решения поставленной задачи будет RMSLE - Root Mean Squared Log Error, которая вычисляется на основе целевой переменной в логарифмическом масштабе. В таком случае целесообразно сразу логарифмировать признак длительности поездки и рассматривать при анализе логарифм в качестве целевого признака:
$$trip\_duration\_log = log(trip\_duration+1),$$
где под символом log подразумевается натуральный логарифм.


In [ ]:
taxi_data['trip_duration_log'] = np.log(taxi_data['trip_duration']+1)

### Задание 3.1.
Постройте гистограмму и коробчатую диаграмму длительности поездок в логарифмическом масштабе (trip_duration_log). 
Исходя из визуализации, сделайте предположение, является ли полученное распределение нормальным? 
Проверьте свою гипотезу с помощью теста Д’Агостино при уровне значимости $\alpha=0.05$. 

а) Чему равен вычисленный p-value? Ответ округлите до сотых.

б) Является ли распределение длительности поездок в логарифмическом масштабе нормальным?

In [ ]:
# ваш код здесь
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(
    rows=1, cols=2,
    vertical_spacing=0.09,
    specs=[[{"type": "Histogram"},{"type": "Box"}]]
    )

fig.add_trace(
    go.Histogram(x=taxi_data['trip_duration_log']),
    row=1, col=1
)
fig.add_trace(
    go.Box(y=taxi_data['trip_duration_log'], boxmean=True),
    row=1, col=2
)

fig.update_layout(
    height=600,
    showlegend=True,
    title_text="<b>Диаграмма длительности поездок в логарифмическом масштабе<b>",
)
fig.show()


In [ ]:
from scipy.stats import normaltest
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
alpha = 0.05

_, p = normaltest(taxi_data['trip_duration_log'])

print('p=%.2f' % p)


if p > alpha/2:
	print(H0)
else:
	print(Ha)

### Задание 3.2.
Постройте визуализацию, которая позволит сравнить распределение длительности поездки в логарифмическом масштабе (trip_duration_log) в зависимости от таксопарка (vendor_id). 

Сравните два распределения между собой.

In [ ]:
# ваш код здесь
fig = px.box(taxi_data, 
             y='trip_duration_log',
             color='vendor_id', 
             title="Распределение длительности поездки в логарифмическом масштабе в зависимости от таксопарка")
fig.show()

Распределения похожи

### Задание 3.3.
Постройте визуализацию, которая позволит сравнить распределение длительности поездки в логарифмическом масштабе (trip_duration_log) в зависимости от признака отправки сообщения поставщику (store_and_fwd_flag). 

Сравните два распределения между собой.

In [ ]:
fig = px.box(taxi_data, 
             y='trip_duration_log',
             color='store_and_fwd_flag', 
             title="Распределение длительности поездки в логарифмическом масштабе в зависимости от признака отправки сообщения поставщику")
fig.show()

Распределения похожи, хотя при отсутствии отметки длительность чуть ниже

### Задание 3.4.
Постройте две визуализации:
* Распределение количества поездок в зависимости от часа дня;
* Зависимость медианной длительности поездки от часа дня.

На основе построенных графиков ответьте на следующие вопросы:

а) В какое время суток такси заказывают реже всего?

б) В какое время суток наблюдается пик медианной длительности поездок?

In [ ]:
plot = taxi_data.groupby('pickup_hour', as_index=False)['id'].count()
plt.figure(figsize=(7, 6))
plt.bar(x=plot['pickup_hour'], height=plot['id'])
plt.xlabel('Pickup Hour')
plt.title('Распределение количества поездок в зависимости от часа дня')
plt.show()

In [ ]:
plot = taxi_data.groupby('pickup_hour', as_index=False)['trip_duration_log'].median()
plt.figure(figsize=(5, 6))
plt.bar(x=plot['pickup_hour'], height=plot['trip_duration_log'])
plt.xlabel('Pickup Hour')
plt.ylabel('Median Trip Duration (Log Scale)')
plt.title('Зависимость медианной длительности поездки от часа дня')
plt.show()

### Задание 3.5.
Постройте две визуализации:
* Распределение количества поездок в зависимости от дня недели;
*  Зависимость медианной длительности поездки от дня недели.

На основе построенных графиков ответьте на следующие вопросы:
а) В какой день недели совершается больше всего поездок?
б) В какой день недели медианная длительность поездок наименьшая?


In [ ]:
# ваш код здесь
plot = taxi_data.groupby('pickup_day_of_week', as_index=False)['id'].count()
plt.figure(figsize=(6, 4))
plt.bar(x=plot['pickup_day_of_week'], height=plot['id'])
plt.xlabel('pickup_day_of_week')
plt.title('Распределение количества поездок в зависимости от дня недели')
plt.show()

In [ ]:
plot = taxi_data.groupby('pickup_day_of_week', as_index=False)['trip_duration_log'].median()
plt.figure(figsize=(6, 4))
plt.bar(x=plot['pickup_day_of_week'], height=plot['trip_duration_log'])
plt.xlabel('Pickup Hour')
plt.ylabel('Median Trip Duration (Log Scale)')
plt.title('Зависимость медианной длительности поездки от дня недели')
plt.show()

### Задание 3.6.
Посмотрим на обе временные характеристики одновременно. 

Постройте сводную таблицу, по строкам которой отложены часы (pickup_hour), по столбцам - дни недели (pickup_day_of_week), а в ячейках - медианная длительность поездки (trip_duration). 

Визуализируйте полученную сводную таблицу с помощью тепловой карты (рекомендуемая палитра - coolwarm).

In [ ]:
# ваш код здесь
pivot_table = taxi_data.pivot_table(values='trip_duration', 
                                    index='pickup_hour', 
                                    columns='pickup_day_of_week', 
                                    aggfunc='median')

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, annot=True, cmap='coolwarm', fmt='g', linewidths=.5)
plt.title('Медианная длительность поездок по часам и дням недели')
plt.xlabel('День недели')
plt.ylabel('Час')
plt.show()

### Задание 3.7.
Постройте две диаграммы рассеяния (scatter-диаграммы):
* первая должна иллюстрировать географическое расположение точек начала поездок (pickup_longitude, pickup_latitude) 
* вторая должна географическое расположение точек завершения поездок (dropoff_longitude, dropoff_latitude).

Для этого на диаграммах по оси абсцисс отложите широту (longitude), а по оси ординат - долготу (latitude). 
Включите в визуализацию только те точки, которые находятся в пределах Нью-Йорка - добавьте следующие ограничения на границы осей абсцисс и ординат:
 
city_long_border = (-74.03, -73.75)

city_lat_border = (40.63, 40.85)

Добавьте на диаграммы расцветку по десяти географическим кластерам (geo_cluster), которые мы сгенерировали ранее. 

**Рекомендация:** для наглядности уменьшите размер точек на диаграмме рассеяния.  


In [ ]:
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)

# ваш код здесь
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.scatter(x=taxi_data['pickup_longitude'], 
            y = taxi_data['pickup_latitude'], 
            c = taxi_data['geo_cluster'],
            s=5 
            )
plt.xlabel('pickup_longitude')
plt.ylabel('pickup_latitude')
plt.xlim(city_long_border)
plt.ylim(city_lat_border)
plt.title('географическое расположение точек начала поездок')

plt.subplot(1, 2, 2)
plt.scatter(x=taxi_data['dropoff_longitude'], 
            y = taxi_data['dropoff_latitude'], 
            c = taxi_data['geo_cluster'],
            s=5 
            )
plt.xlabel('dropoff_longitude')
plt.ylabel('dropoff_latitude')
plt.xlim(city_long_border)
plt.ylim(city_lat_border)
cbar = plt.colorbar()
cbar.set_label('geo_cluster')
plt.title('географическое расположение точек завершения поездок')
plt.show()

plt.tight_layout()
cbar = plt.colorbar()
cbar.set_label('geo_cluster')
plt.show()

## 4. Отбор и преобразование признаков

Перед тем как перейти к построению модели, осталось сделать ещё несколько шагов.
* Следует помнить, что многие алгоритмы машинного обучения не могут обрабатывать категориальные признаки в их обычном виде. Поэтому нам необходимо их закодировать;
* Надо отобрать признаки, которые мы будем использовать для обучения модели;
*  Необходимо масштабировать и трансформировать некоторые признаки для того, чтобы улучшить сходимость моделей, в основе которых лежат численные методы.


In [ ]:
print('Shape of data: {}'.format(taxi_data.shape))
print('Columns: {}'.format(taxi_data.columns))

Для удобства работы сделаем копию исходной таблицы с поездками:

In [ ]:
taxi_data.drop(columns=['cluster_squared_distance','average speed'], 
               inplace = True)

In [ ]:
train_data = taxi_data.copy()
train_data.head()

### Задание 4.1.
Сразу позаботимся об очевидных неинформативных и избыточных признаках. 

а) Какой из признаков является уникальным для каждой поездки и не несет полезной информации в определении ее продолжительности?

б) Утечка данных (data leak) - это…

в) Подумайте, наличие какого из признаков в обучающем наборе данных создает утечку данных?
В наших данных утечку создаёт признак dropoff_datetime — зафиксированное время остановки счётчика. Начиная поездку, мы никак не можем точно знать, когда она закончится, ведь мы как раз и пытаемся предсказать время окончания поездки.

г) Исключите выбранные в пунктах а) и в) признаки из исходной таблицы с данными. Сколько столбцов в таблице у вас осталось?


In [ ]:
# ваш код здесь
train_data.drop(columns=['id','dropoff_datetime'], 
               inplace = True)
train_data.shape

Ранее мы извлекли всю необходимую для нас информацию из даты начала поездки, теперь мы можем избавиться от этих признаков, так как они нам больше не понадобятся:


In [ ]:
drop_columns = ['pickup_datetime', 'pickup_date']
train_data = train_data.drop(drop_columns, axis=1)
print('Shape of data:  {}'.format(train_data.shape))

### Задание 4.2.

Закодируйте признак vendor_id в таблице train_data таким образом, чтобы он был равен 0, если идентификатор таксопарка равен 1, и 1 — в противном случае.

Закодируйте признак store_and_fwd_flag в таблице train_data таким образом, чтобы он был равен 0, если флаг выставлен в значение 'N', и 1 — в противном случае.

а) Рассчитайте среднее по закодированному столбцу vendor_id. Ответ приведите с точностью до сотых.

б) Рассчитайте среднее по закодированному столбцу store_and_fwd_flag. Ответ приведите с точностью до тысячных.



In [ ]:
train_data['vendor_id'] = train_data['vendor_id'].replace({1:0, 2:1})
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].replace({'N':0, 'Y':1})
train_data

In [ ]:
#а) Рассчитайте среднее по закодированному столбцу vendor_id. Ответ приведите с точностью до сотых.
print(round(train_data['vendor_id'].mean(),2))
#б) Рассчитайте среднее по закодированному столбцу store_and_fwd_flag. Ответ приведите с точностью до тысячных.
print(round(train_data['store_and_fwd_flag'].mean(),3))

### Задание 4.3.
Создайте таблицу data_onehot из закодированных однократным кодированием признаков pickup_day_of_week, geo_cluster и events в таблице train_data с помощью OneHotEncoder из библиотеки sklearn. Параметр drop выставите в значение 'first', чтобы удалять первый бинарный столбец, тем самым не создавая излишних признаков.

В результате работы OneHotEncoder вы получите безымянный numpy-массив, который нам будет необходимо преобразовать обратно в DataFrame, для более удобной работы в дальнейшем. Чтобы получить имена закодированных столбцов у объекта типа OneHotEncoder есть специальный метод get_feature_names_out(). Он возвращает список новых закодированных имен столбцов в формате <оригинальное имя столбца>_<имя категории>.

Пример использования:

``` python
# Получаем закодированные имена столбцов
column_names = one_hot_encoder.get_feature_names_out()
# Составляем DataFrame из закодированных признаков
data_onehot = pd.DataFrame(data_onehot, columns=column_names)
```

В этом псевдокоде:
* one_hot_encoder - объект класса OneHotEncoder
* data_onehot - numpy-массив, полученный в результате трансформации кодировщиком

В результате выполнения задания у вас должен быть образован DataFrame `data_onehot`, который содержит кодированные категориальные признаки pickup_day_of_week, geo_cluster и events. 


Сколько бинарных столбцов у вас получилось сгенерировать с помощью однократного кодирования?


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output=False)
data_onehot = encoder.fit_transform(train_data[['pickup_day_of_week', 'geo_cluster','events']])
column_names = encoder.get_feature_names_out()
print(column_names)
data_onehot = pd.DataFrame(data_onehot, columns=column_names)
display(data_onehot.head())
display(data_onehot.info())

Добавим полученную таблицу с закодированными признаками:

In [ ]:
train_data = pd.concat(
    [train_data.reset_index(drop=True).drop(['pickup_day_of_week', 'geo_cluster','events'], axis=1), data_onehot], 
    axis=1
)
print('Shape of data: {}'.format(train_data.shape))

Теперь, когда категориальные признаки предобработаны, сформируем матрицу наблюдений X, вектор целевой переменной y и его логарифм y_log. В матрицу наблюдений войдут все столбцы из таблицы с поездками за исключением целевого признака trip_duration и его логарифмированной версии trip_duration_log:


In [ ]:
X = train_data.drop(['trip_duration', 'trip_duration_log'], axis=1)
y = train_data['trip_duration']
y_log = train_data['trip_duration_log']

Все наши модели мы будем обучать на логарифмированной версии y_log. 

Выбранный тип валидации - hold-out. Разобьем выборку на обучающую и валидационную в соотношении 67/33:

In [ ]:
X_train, X_valid, y_train_log, y_valid_log = model_selection.train_test_split(
    X, y_log, 
    test_size=0.33, 
    random_state=42
)

На данный момент у нас достаточно много признаков: скорее всего, не все из них будут важны. Давайте оставим лишь те, которые сильнее всего связаны с целевой переменной и точно будут вносить вклад в повышение качества модели.


### Задание 4.4.
С помощью SelectKBest отберите 25 признаков, наилучшим образом подходящих для предсказания целевой переменной в логарифмическом масштабе. Отбор реализуйте по обучающей выборке, используя параметр score_func = f_regression.

Укажите признаки, которые вошли в список отобранных


In [ ]:
# ваш код здесь
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=25)
selector.fit(X_train, y_train_log)
 
cols = selector.get_feature_names_out()

In [ ]:
X_train, X_valid = X_train[cols], X_valid[cols]
print(X_train.shape, X_valid.shape)

Так как мы будем использовать различные модели, в том числе внутри которых заложены численные методы оптимизации, то давайте заранее позаботимся о масштабировании факторов. 


### Задание 4.5.
Нормализуйте предикторы в обучающей и валидационной выборках с помощью MinMaxScaler из библиотеки sklearn. Помните, что обучение нормализатора производится на обучающей выборке, а трансформация на обучающей и валидационной!

Рассчитайте среднее арифметическое для первого предиктора (т. е. для первого столбца матрицы) из валидационной выборки. Ответ округлите до сотых.


In [ ]:
# ваш код здесь
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns= X_train.columns)
X_valid_scaled = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)
X_valid_scaled.head(2)

In [ ]:
round(X_valid_scaled['vendor_id'].mean(),2)

In [ ]:
X_train_scaled.info()

In [ ]:
X_valid_scaled.info()

## 5. Решение задачи регрессии: линейная регрессия и деревья решений

Определим метрику, по которой мы будем измерять качество наших моделей. Мы будем следовать канонам исходного соревнования на Kaggle и в качестве метрики использовать RMSLE (Root Mean Squared Log Error), которая вычисляется как:
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n(log(y_i+1)-log(\hat{y_i}+1))^2},$$
где:
* $y_i$ - истинная длительность i-ой поездки на такси (trip_duration)
* $\hat{y_i}$- предсказанная моделью длительность i-ой поездки на такси

Заметим, что логарифмирование целевого признака мы уже провели заранее, поэтому нам будет достаточно вычислить метрику RMSE для модели, обученной прогнозировать длительность поездки такси в логарифмическом масштабе:
$$z_i=log(y_i+1),$$
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n(z_i-\hat{z_i})^2}=\sqrt{MSE(z_i,\hat{z_i})}$$ 



### Задание 5.1.
Постройте модель линейной регрессии на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию.

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
# ваш код здесь
lr_model = linear_model.LinearRegression()
lr_model.fit(X_train_scaled, y_train_log)

y_train_predict = lr_model.predict(X_train_scaled)
y_valid_predict = lr_model.predict(X_valid_scaled)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)


### Задание 5.2.
Сгенерируйте полиномиальные признаки 2-ой степени с помощью PolynomialFeatures из библиотеки sklearn. Параметр include_bias выставите в значение False.

Постройте модель полиномиальной регрессии 2-ой степени на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию.

а) Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.

б) Наблюдаются ли у вашей модели признаки переобучения?


In [ ]:
# ваш код здесь
poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train_scaled)
X_train_scaled_poly= poly.transform(X_train_scaled)
X_valid_scaled_poly = poly.transform(X_valid_scaled)

lr_model_poly = linear_model.LinearRegression()
lr_model_poly.fit(X_train_scaled_poly, y_train_log)

y_train_predict = lr_model_poly.predict(X_train_scaled_poly)
y_valid_predict = lr_model_poly.predict(X_valid_scaled_poly)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)

### Задание 5.3.
Постройте модель полиномиальной регрессии 2-ой степени с L2-регуляризацией (регуляризация по Тихонову) на обучающей выборке  (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Коэффициент регуляризации $\alpha$ установите равным 1, остальные параметры оставьте по умолчанию.

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
# ваш код здесь
lr_model_poly_r = linear_model.Ridge(alpha=1)
lr_model_poly_r.fit(X_train_scaled_poly, y_train_log)

y_train_predict = lr_model_poly_r.predict(X_train_scaled_poly)
y_valid_predict = lr_model_poly_r.predict(X_valid_scaled_poly)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)

### Задание 5.4.
Постройте модель дерева решений (DecisionTreeRegressor) на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). Все параметры оставьте по умолчанию. 

а) Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.

б) Наблюдаются ли у вашей модели признаки переобучения?


In [ ]:
# ваш код здесь
dt = tree.DecisionTreeRegressor(random_state=42)
dt.fit(X_train_scaled, y_train_log)

y_train_predict = dt.predict(X_train_scaled)
y_valid_predict = dt.predict(X_valid_scaled)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)

### Задание 5.5.
Переберите все возможные варианты глубины дерева решений в диапазоне от 7 до 20:

max_depths = range(7, 20)

Параметр random_state задайте равным 42.

Постройте линейные графики изменения метрики RMSE на тренировочной и валидационной выборках в зависимости от значения параметра глубины дерева решений. 

а) Найдите оптимальное значение максимальной глубины дерева, для которой будет наблюдаться минимальное значение RMSLE на обучающей выборке, но при этом еще не будет наблюдаться переобучение (валидационная кривая еще не начинает возрастать).

б) Чему равно значение метрик RMSLE на тренировочной и валидационной выборках для дерева решений с выбранной оптимальной глубиной? Ответ округлите до сотых.


In [ ]:
# ваш код здесь
import optuna
def optuna_dt(trial):
  max_depth = trial.suggest_int('max_depth', 7, 20, 1)
  model = tree.DecisionTreeRegressor(random_state=42,max_depth=max_depth)
  model.fit(X_train_scaled, y_train_log)
  y_train_predict = model.predict(X_train_scaled)
  score = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
  return score

In [ ]:
study = optuna.create_study(study_name="DecisionTreeRegressor", direction="minimize")
study.optimize(optuna_dt, n_trials=10)
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("rmsle на обучающем наборе: {:.2f}".format(study.best_value))

In [ ]:
model = tree.DecisionTreeRegressor(max_depth=19,random_state=42)
model.fit(X_train_scaled, y_train_log)
y_train_predict = model.predict(X_train_scaled)
y_valid_predict = model.predict(X_valid_scaled)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)

In [ ]:
dep = []
er_train = []
er_valid = []
for i in range(7,20):
    model = tree.DecisionTreeRegressor(max_depth=i,random_state=42)
    model.fit(X_train_scaled, y_train_log)
    y_train_predict = model.predict(X_train_scaled)
    y_valid_predict = model.predict(X_valid_scaled)
    rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
    rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
    dep.append(i)
    er_train.append(rmsle_train)
    er_valid.append(rmsle_valid)           

In [ ]:
fig, ax = plt.subplots()

ax.plot(dep, er_train, label='rmsle_train', color='blue')

ax.plot(dep, er_valid, label='rmsle_valid', color='red')

ax.set_xlabel('rmsle_train')
ax.set_ylabel('rmsle_valid')
ax.set_title('RMSLE')

plt.show()

## 6. Решение задачи регрессии: ансамблевые методы и построение прогноза

Переходим к тяжелой артиллерии: ансамблевым алгоритмам. 

### Задание 6.1.

Постройте модель случайного леса на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). В качестве гиперпараметров укажите следующие:
* n_estimators=200,
* max_depth=12,
* criterion='squared_error',
* min_samples_split=20,
* random_state=42

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
# ваш код здесь
rf = ensemble.RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    criterion='squared_error',
    min_samples_split=20,
    random_state=42
)
rf.fit(X_train_scaled, y_train_log)

y_train_predict = rf.predict(X_train_scaled)
y_valid_predict = rf.predict(X_valid_scaled)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)


### Задание 6.2.
Постройте модель градиентного бустинга над деревьями решений (GradientBoostingRegressor) на обучающей выборке (факторы должны быть нормализованы, целевую переменную используйте в логарифмическом масштабе). В качестве гиперпараметров укажите следующие:
* learning_rate=0.5,
* n_estimators=100,
* max_depth=6, 
* min_samples_split=30,
* random_state=42

Для полученной модели рассчитайте метрику RMSLE на тренировочной и валидационной выборках. Ответ округлите до сотых.


In [ ]:
# ваш код здесь
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(
    learning_rate=0.5,
    n_estimators=100,
    max_depth=6, 
    min_samples_split=30,
    random_state=42
)
gb.fit(X_train_scaled, y_train_log)

y_train_predict = gb.predict(X_train_scaled)
y_valid_predict = gb.predict(X_valid_scaled)

rmsle_train = round(metrics.mean_squared_error(y_train_log, y_train_predict)**0.5,2)
rmsle_valid = round(metrics.mean_squared_error(y_valid_log, y_valid_predict)**0.5,2)
print('train:',rmsle_train,'\n',
      'valid:',rmsle_valid)

### Задание 6.3.
Какая из построенных вами моделей показала наилучший результат (наименьшее значение RMSLE на валидационной выборке)?
* Линейная регрессия
* Полиномиальная регрессия 2ой степени
* Дерево решений
* Случайный лес
* Градиентный бустинг над деревьями решений


answer:Градиентный бустинг над деревьями решений

### Задание 6.4.
Постройте столбчатую диаграмму коэффициентов значимости каждого из факторов.

Укажите топ-3 наиболее значимых для предсказания целевого признака - длительности поездки в логарифмическом масштабе - факторов.


In [ ]:
# ваш код здесь
fig, ax = plt.subplots(figsize=(13, 5)) 
feature = X_train_scaled.columns 
feature_importances = gb.feature_importances_
sns.barplot(x=feature, y=feature_importances, ax=ax)
ax.set_title('Bar plot feature importances')
ax.set_xlabel('Features')
ax.set_ylabel('Importances')
plt.xticks(rotation=45, ha='right')
fig.show()

### Задание 6.5.
Для лучшей из построенных моделей рассчитайте медианную абсолютную ошибку (MeAE - в sklearn функция median_absolute_error) предсказания длительности поездки такси на валидационной выборке:
$$ MeAE = median(|y_i-\hat{y_i}|)$$

Значение метрики MeAE переведите в минуты и округлите до десятых.


In [ ]:
# ваш код здесь
y_train, y_valid = model_selection.train_test_split(
    y, 
    test_size=0.33, 
    random_state=42
)
gb.fit(X_train_scaled, y_train)
y_train_predict = gb.predict(X_train_scaled)
y_valid_predict = gb.predict(X_valid_scaled)
med_ae_train = round(metrics.median_absolute_error(y_train, y_train_predict)/60,1)
med_ae_valid = round(metrics.median_absolute_error(y_valid, y_valid_predict)/60,1)
print('train:',med_ae_train,'\n',
      'valid:',med_ae_valid)

Финальный шаг - сделать submit -  предсказание для отложенного тестового набора данных. 

Прочитаем тестовые данные и заранее выделим столбец с идентификаторами поездок из тестового набора данных. Он нам еще пригодится:


In [ ]:
test_data = pd.read_csv("/Users/Anna/Study/skillfactory/practice/math/Project5_test_data.csv")
osrm_data_test = pd.read_csv("/Users/Anna/Study/skillfactory/practice/math/Project5_osrm_data_test.csv")
test_id = test_data['id']

Перед созданием прогноза для тестовой выборки необходимо произвести все манипуляции с данными, которые мы производили с тренировочной выборкой, а именно:
* Перевести признак pickup_datetime в формат datetime;
* Добавить новые признаки (временные, географические, погодные и другие факторы);
* Произвести очистку данных от пропусков;
* Произвести кодировку категориальных признаков:
    * Закодировать бинарные признаки;
    * Закодировать номинальные признаки с помощью обученного на тренировочной выборке OneHotEncoder’а;
* Сформировать матрицу наблюдений, оставив в таблице только те признаки, которые были отобраны с помощью SelectKBest;
* Нормализовать данные с помощью обученного на тренировочной выборке MinMaxScaler’а.


In [ ]:
test_data['pickup_datetime']=pd.to_datetime(test_data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S')
test_data['pickup_date'] = test_data['pickup_datetime'].dt.date
test_data['pickup_date'] = pd.to_datetime(test_data['pickup_date'])

In [ ]:
test_data = add_datetime_features(test_data)
test_data = add_holiday_features(test_data, holiday_data)
test_data = add_osrm_features(test_data, osrm_data_test)
test_data = add_geographical_features(test_data)
test_data = add_cluster_features(test_data, kmeans)
test_data = add_weather_features(test_data, weather_data)
display(test_data.head())
display(test_data.info())

In [ ]:
test_data.isna().sum()

In [ ]:
def fill_null_weather_data(df):
   value = {'temperature':df.groupby('pickup_date')['temperature'].transform('mean'),
       'visibility':df.groupby('pickup_date')['visibility'].transform('mean'),
       'wind speed':df.groupby('pickup_date')['wind speed'].transform('mean'),
        'precip':df.groupby('pickup_date')['precip'].transform('mean'),
        'events':'None'}
   df.fillna(value, inplace=True)
   return df

In [ ]:
test_data = fill_null_weather_data(test_data)
display(test_data.head())
display(test_data.info())
display(test_data.isna().sum())

In [ ]:
test_data['vendor_id'] = test_data['vendor_id'].apply(lambda x: 0 if x == 1 else 1)
test_data['store_and_fwd_flag'] = test_data['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)

test_data_onehot = encoder.fit_transform(test_data[['pickup_day_of_week', 'geo_cluster','events']])
column_names = encoder.get_feature_names_out()
test_data_onehot = pd.DataFrame(test_data_onehot, columns=column_names)
test_data = pd.concat(
    [test_data.reset_index(drop=True).drop(['pickup_day_of_week', 'geo_cluster','events'], axis=1), test_data_onehot], 
    axis=1
)
display(test_data.head(2))
display(test_data.info())

In [ ]:
X_test = test_data[cols]
X_test_scaled = scaler.transform(X_test)
print('Shape of data: {}'.format(X_test_scaled.shape))

Только после выполнения всех этих шагов можно сделать предсказание длительности поездки для тестовой выборки. Не забудьте перевести предсказания из логарифмического масштаба в истинный, используя формулу:
$$y_i=exp(z_i)-1$$

После того, как вы сформируете предсказание длительности поездок на тестовой выборке вам необходимо будет создать submission-файл в формате csv, отправить его на платформу Kaggle и посмотреть на результирующее значение метрики RMSLE на тестовой выборке.

Код для создания submission-файла:


In [ ]:
print('Shape of data: {}'.format(X_train_scaled.shape))

In [ ]:
gb.fit(X_train_scaled, y_train)

y_test_predict = gb.predict(X_test_scaled)

In [ ]:
# ваш код здесь
submission = pd.DataFrame({'id': test_id, 'trip_duration': y_test_predict})
submission.to_csv('/Users/Anna/Study/skillfactory/practice/math/submission_gb.csv', index=False)

### **В качестве бонуса**

В завершение по ансамблевым мы предлагаем вам попробовать улучшить свое предсказание, воспользовавшись моделью экстремального градиентного бустинга (XGBoost) из библиотеки xgboost.

**XGBoost** - современная модель машинного обучения, которая является продолжением идеи градиентного бустинга Фридмана. У нее есть несколько преимуществ по сравнению с классической моделью градиентного бустинга из библиотеки sklearn: повышенная производительность путем параллелизации процесса обучения, повышенное качество решения за счет усовершенствования алгоритма бустинга, меньшая склонность к переобучению и широкий функционал возможности управления параметрами модели.


Для ее использования необходимо для начала установить пакет xgboost:

In [ ]:
pip install xgboost

После чего модуль можно импортировать:

In [ ]:
import xgboost as xgb

Перед обучением модели необходимо перевести наборы данных в тип данных xgboost.DMatrix:

In [ ]:
cols

In [ ]:
# Создание матриц наблюдений в формате DMatrix
dtrain = xgb.DMatrix(X_train_scaled, label=y_train_log, feature_names=list(cols))
dvalid = xgb.DMatrix(X_valid_scaled, label=y_valid_log, feature_names=list(cols))
dtest = xgb.DMatrix(X_test_scaled, feature_names=list(cols))

Обучение модели XGBoost происходит с помощью метода train, в который необходимо передать параметры модели, набор данных, количество базовых моделей в ансамбле, а также дополнительные параметры:


In [ ]:
# Гиперпараметры модели
xgb_pars = {'min_child_weight': 20, 'eta': 0.1, 'colsample_bytree': 0.9, 
            'max_depth': 6, 'subsample': 0.9, 'lambda': 1, 'nthread': -1, 
            'booster' : 'gbtree', 'eval_metric': 'rmse', 'objective': 'reg:squarederror'
           }
# Тренировочная и валидационная выборка
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
# Обучаем модель XGBoost
model = xgb.train(
    params=xgb_pars, #гиперпараметры модели
    dtrain=dtrain, #обучающая выборка
    num_boost_round=300, #количество моделей в ансамбле
    evals=watchlist, #выборки, на которых считается матрица
    early_stopping_rounds=20, #раняя остановка
    maximize=False, #смена поиска максимума на минимум
    verbose_eval=10 #шаг, через который происходит отображение метрик
)

Предсказать целевой признак на новых данных можно с помощью метода predict():

In [ ]:
#Делаем предсказание на тестовом наборе данных
y_test_predict = np.exp(model.predict(dtest)) - 1
print('Modeling RMSLE %.5f' % model.best_score)

Также как и все модели, основанные на использовании деревьев решений в качестве базовых моделей, XGBoost имеет возможность определения коэффициентов важности факторов. Более того, в библиотеку встроена возможность визуализации важность факторов в виде столбчатой диаграммы. За эту возможность отвечает функция plot_importance():


In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
xgb.plot_importance(model, ax = ax, height=0.5)